In [1]:
import csv
import numpy as np

labeledin_dailymed_ids = []
ids_dailymed_ids = []

def unique(a):
    """ return the list with duplicate elements removed """
    return list(set(a))

def intersect(a, b):
    """ return the intersection of two lists """
    return list(set(a) & set(b))

def union(a, b):
    """ return the union of two lists """
    return list(set(a) | set(b))

with open('labeledin_normal.dsv') as dsvfile:
    reader = csv.reader(dsvfile, delimiter='|')
    for row in reader:
        labeledin_dailymed_ids.append(row[1])

with open('labeledin_crowd.dsv') as dsvfile:
    reader = csv.reader(dsvfile, delimiter='|')
    for row in reader:
        labeledin_dailymed_ids.append(row[1])

with open('ManualCuration+DrugCentral.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        ids_dailymed_ids.append(row[3].rstrip())     
        
print('labeledin size: ' + str(len(labeledin_dailymed_ids)))
print('labeledin unique size: ' + str(len(unique(labeledin_dailymed_ids))))

unique_labeledin_dmids = unique(labeledin_dailymed_ids)

print('ids size: ' + str(len(ids_dailymed_ids)))
print('ids unique size: ' + str(len(unique(ids_dailymed_ids))))

unique_ids_dmids = unique(ids_dailymed_ids)

intersection_dmids = intersect(unique_labeledin_dmids, unique_ids_dmids)
#print(intersection_dmids)
#print('intersection unique size: ' + str(len(intersection_dmids)))

labeledin_intersection_data = []

with open('labeledin_normal.dsv') as dsvfile:
    reader = csv.reader(dsvfile, delimiter='|')
    for row in reader:
        if (row[1] in intersection_dmids):
            #print('n')
            row_tmp = []
            row_tmp.append(row[1].replace(" ", ""))
            row_tmp.append(row[2].replace(" ", ""))
            labeledin_intersection_data.append(row_tmp)

with open('labeledin_crowd.dsv') as dsvfile:
    reader = csv.reader(dsvfile, delimiter='|')
    for row in reader:
        if (row[1] in intersection_dmids):
            #print('c')
            row_tmp = []
            row_tmp.append(row[1])
            row_tmp.append(row[2])
            labeledin_intersection_data.append(row_tmp)
            
ids_data = []
            
with open('ManualCuration+DrugCentral.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        tmp = row[3].rstrip()
        if (tmp in intersection_dmids):
            row_tmp = []
            row_tmp.append(row[3].rstrip())
            tmp_16 = row[16].rstrip()
            tmp_16 = tmp_16.replace(",",";")
            row_tmp.append(tmp_16)
            tmp_17 = row[17].rstrip()
            tmp_17 = tmp_17.replace(",",";")
            row_tmp.append(tmp_17)
            ids_data.append(row_tmp)
            
print()
print("IDS Data: ")
print()
print(ids_data)
    
print()
print("LabeledIn Data: ")
print()
print(labeledin_intersection_data)




labeledin size: 3504
labeledin unique size: 1206
ids size: 152
ids unique size: 141

IDS Data: 

[['d6a161d9-7e7a-4711-ba0e-c9040c42268d', '', 'C0038525'], ['1521d321-e724-4ffc-adad-34bf4f44fac7', 'C0026764', 'C0026764;C0334634'], ['ec476b11-0b3c-4139-b1eb-a3daa76bc271', 'C0311221;C0003873;C0033860;C0374079;C0278511;C0729502;C0024305;C0242379;C0684249;C0026948;C1168401;C0006142;C1458155;C0153555;C0677908;C0948840;C0023449;C0008493;C1396879;C0349557', 'C0033860;C3163805;C0278996;C3495559;C1135868;C0026948;C0024305;C0023449;C0242379;C0006413;C0003873;C0585442'], ['939b5d1f-9fb2-4499-80ef-0607aa6b114e', 'C1514428;C3897738;C0278689;C0302592;C0002199;C0278678;C0017636;C4083276;C0948380', 'C0346974;C2585890;C1621958;C0278678;C0302592;C0153467;C0346161;C0153579'], ['f5a860f3-37ec-429c-ae04-9c88d7c55c08', 'C0812413;C0278940;C0814469;C4083276', 'C0812413;C0007131'], ['f5a860f3-37ec-429c-ae04-9c88d7c55c08', 'C0812413;C0278940;C0814469;C4083276', 'C0042345'], ['8bc6397e-4bd8-4d37-a007-a327e4da34d

In [2]:
import requests
from xmljson import badgerfish as bf
from json import dumps
from xml.etree.ElementTree import fromstring
import xml.etree.ElementTree as et

def getLabeledInCUIs(dmid):
    for item in labeledin_intersection_data:
        if (item[0] == dmid):
            return item[1]
    return ''

def intersection(first, *others):
    return set(first).intersection(*others)

def makeList(item):
    return item.split(";")

def testMethod(dmid):
    print(dmid)
    r = requests.get('https://dailymed.nlm.nih.gov/dailymed/services/v2/spls/'+
                     dmid+'.xml')
    #print(r.status_code)
    #print(r.headers['content-type'])
    #print(r.encoding)
    tmp = et.fromstring(r.text)
    #root = tmp.getroot()
    for medicineName in tmp.findall('.//{urn:hl7-org:v3}genericMedicine'):
        for tmp2 in medicineName.findall("{urn:hl7-org:v3}name"):
            return tmp2.text
            #break
        #break
        
#     print(tmp.tag)
   
with open('LabeledIn_comparison_results_table.csv', 'w', newline='') as csvfile:
    fieldnames = ['drug_name', 'dailymed_id', 'ids_cuis', 'drugcentral_cuis', 'labeledin_cuis', '#ids_cuis', '#dc_cuis',
                  '#li_cuis', 'full_intersection', 'ids_dc', 'ids_li', 'li_dc', '#full_intersection', '#ids_dc', '#ids_li', '#li_dc']
    writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(fieldnames)
    dailymed_ids_addressed = []
    
    for item in ids_data:
        if (item[0] not in dailymed_ids_addressed):
            # Extract CUIs
            row = []
            dailymed_id = item[0]
            dailymed_ids_addressed.append(dailymed_id)
            drug_name = testMethod(dailymed_id)

            ids_cuis = item[1]
            drugcentral_cuis = item[2]
            labeledin_cuis = getLabeledInCUIs(dailymed_id)

            # Compute intersections
            all_intersection = intersection(makeList(ids_cuis), makeList(drugcentral_cuis), makeList(labeledin_cuis))
            ids_dc = intersection(makeList(ids_cuis), makeList(drugcentral_cuis))
            ids_li = intersection(makeList(ids_cuis), makeList(labeledin_cuis))
            li_dc = intersection(makeList(labeledin_cuis), makeList(drugcentral_cuis))

            # Construct row
            row.append(drug_name)
            row.append(dailymed_id)

            row.append(ids_cuis)
            row.append(drugcentral_cuis)
            row.append(labeledin_cuis)

            row.append(len(makeList(ids_cuis)))
            row.append(len(makeList(drugcentral_cuis)))
            row.append(len(makeList(labeledin_cuis)))

            row.append(';'.join(all_intersection))
            row.append(';'.join(ids_dc))
            row.append(';'.join(ids_li))
            row.append(';'.join(li_dc))
            row.append(len(all_intersection))
            row.append(len(ids_dc))
            row.append(len(ids_li))
            row.append(len(li_dc))

            # Write row
            writer.writerow(row)

            #writer.writerow({'first_name': 'Lovely', 'last_name': 'Spam'})
            #writer.writerow({'first_name': 'Wonderful', 'last_name': 'Spam'})

d6a161d9-7e7a-4711-ba0e-c9040c42268d
1521d321-e724-4ffc-adad-34bf4f44fac7
ec476b11-0b3c-4139-b1eb-a3daa76bc271
939b5d1f-9fb2-4499-80ef-0607aa6b114e
f5a860f3-37ec-429c-ae04-9c88d7c55c08
8bc6397e-4bd8-4d37-a007-a327e4da34d9
7fa41601-7fb5-4155-8e50-2ae903f0d2d6
b50667e4-5ebc-4968-a646-d605058dbef0
b172773b-3905-4a1c-ad95-bab4b6126563
492dbdb2-077e-4064-bff3-372d6af0a7a2
a313c111-e539-47bc-9d57-c3767f74bcca
